In [3]:
from API_Keys.api_keys import MyKeys
from googleapiclient.discovery import build

In [4]:
keys = MyKeys()
youtube = build('youtube', 'v3', developerKey = keys.api_key)

In [38]:
request = youtube.commentThreads().list(
    part = 'snippet',
    videoId = 'tgB1wUcmbbw',
    maxResults = 100
)

response = request.execute()

In [46]:
response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'k8MlCg5Q2RBeFIOkPh-5UKB13oA',
 'nextPageToken': 'QURTSl9pMFY1RThFRmhBNzBDRmxpLWJxcVM0cmpfNGtrWFBfcWRlc2VuQlYyT1VzaS1kcnRGMHlFbHIySmFWb0hnZS1HT2VlcFRFVFFwQQ==',
 'pageInfo': {'totalResults': 94, 'resultsPerPage': 100},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'RDWTUWYxUje37suxgHss8Mpl0P0',
   'id': 'Ugy90Fs6MI9kyITohiJ4AaABAg',
   'snippet': {'videoId': 'tgB1wUcmbbw',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'i8eJGLt_J7j45xt3Y85KlzOueZc',
     'id': 'Ugy90Fs6MI9kyITohiJ4AaABAg',
     'snippet': {'videoId': 'tgB1wUcmbbw',
      'textDisplay': 'theres less then 60 days until the movies is coming and yet all we have is a teaser',
      'textOriginal': 'theres less then 60 days until the movies is coming and yet all we have is a teaser',
      'authorDisplayName': 'Benjamin Anthony',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AKedOLTztPLnHH9PmjrxOsMA5SCQJQwjoNzqeuVq82npVw=s48-c-k-c0x

In [45]:
for elem in response['items']:
    print(elem['snippet']['topLevelComment']['snippet']['authorChannelId']['value'])

UC3njorgHwffv7iAj5bpt47g
UC0C58OTlCpi783HfLnnImzg
UCVGTfb_BclClhWRevX6FdFw
UCvJg04OOEU4x7UVxviSjVjw
UCVsTDpfSjYqE0EtAxgmN2IQ
UC2IpXuKfFyAe5uhgy8tnWMw
UCtTv3S8A6TnQ6NzshEF4uBg
UCR0KlyAM0nKlOJhvorFYsTw
UCGJk0y_FG22JQl4Q1WzV9lw
UCGKWvte1HefvEPkWaa64TOQ
UClCeVE7x-hieekG5QaytZBA
UCdjW-pbz_KJco0TExZlAprg
UCbn925pBQbNMYshX0O6KbCA
UCvPQJFaoGsSblZNjfhrCBhQ
UCb8jEhwp0LVTg85JjLCqk5A
UCfW3mhfm3a811nAUaJhRh2w
UCfW3mhfm3a811nAUaJhRh2w
UCYlVm2jfelidg43QAKByRcw
UC3R8B7g8IOiMwU0A9N5rpjg
UCa-dNU7oCWhpzeHYI9Ht2Fw
UCKKm2hUduZE0w93dB3t4OLg
UCYInrrkUNpuVaDRzXP7mX7A
UCXe5rjEFf1HSGKHNkZUhGmw
UCGl1XphfI5Nqq6xJC4fgI3w
UCBSEe38My27qEwiOoJ5JH9Q
UCjqyn9mOH86KKZaecD323EQ
UC-H0qd7pRRPGODngACRI2xg
UCXjcWF-Lquc98YSFmQqnhHw
UCEEb6tewOO9SLaOqeYl3Hqg
UCdCCE9SUWWoDky5nOFIPnHw
UCLdi18_glFV-lGyNW97fqUA
UCPiuYEcEMmu6sy2AnwRrdLg
UCBMKlDq_Jx5SkoDDppr4kDA
UCC2K6sUmKU--rbNY3LcVudw
UCapstxi7lNnsqYMTzbxuQWg
UCuW-Vh1uzr_ESIT_NC_tO0A
UCUX2IZZZYPBUl8_zK51EsJA
UCiuEVisG2jn0cVUVBIYp8bA
UCg3PL3N9dvVCYztP5hhVA8Q
UCfdhu-VKC22E-ZS_nKlS5nA
